In [ ]:
import pandas as pd
import datetime

In [ ]:
# load in csv
damage_reports = pd.read_csv("https://raw.githubusercontent.com/data-challengers/DC5/main/DC5-Data/Damage%20Reports/mc1-reports-data.csv")

# convert timevariable to datetime
damage_reports['time'] = pd.to_datetime(damage_reports['time'])

In [ ]:
# round to hour for scroll
damage_reports['time'] = damage_reports['time'].round('H')


In [ ]:
# import the background image
import base64
with open("C:/Users/rocio/Desktop/datachallenges/DC5/DC5-data-revised/Sensor Data and Maps/StHimarkLabeledMap.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()
#add the prefix that plotly will want when using the string as source
encoded_image = "data:image/png;base64," + encoded_string

In [ ]:
# Define a function to map the values
def set_value(location, assigned_value):
    return assigned_value[location]
  
# Create the dictionary
neighborhood_x = {1: 18, 
                  2: 30, 
                  3: 50,
                  4: 69,
                  5: 30, 
                  6: 30, 
                  7: 98,
                  8: 82,
                  9: 60,
                  10: 76, 
                  11: 85,
                  12: 85,
                  13: 75,
                  14: 48,
                  15: 39, 
                  16: 40,
                  17: 58,
                  18: 60,
                  19: 46}

 
neighborhood_y = {1: 70,
                2: 75,
                3: 80,
                4: 70,
                5: 46,
                6: 60,
                7: 40,
                8: 15,
                9: 20,
                10: 23,
                11: 25,
                12: 42,
                13: 45,
                14: 63,
                15: 64,
                16: 52,
                17: 35,
                18: 50,
                19: 49
}

# Add column for X
damage_reports['X'] = damage_reports['location'].apply(set_value, args =(neighborhood_x, ))

# Add column for Y
damage_reports['Y'] = damage_reports['location'].apply(set_value, args =(neighborhood_y, ))

In [ ]:
import random

# dodge the X and Y axes a little for visibility
def toggle(coordinate):
    return coordinate + random.randrange(-4, 4)

damage_reports['X'] = damage_reports['X'].apply(toggle)
damage_reports['Y'] = damage_reports['Y'].apply(toggle)


In [ ]:
damage_reports.fillna(0, inplace = True)

In [ ]:
damage_reports

In [ ]:
damage_reports_samples = damage_reports[damage_reports['time'] <= '2020-04-06 02:50:00	']

In [ ]:
damage_reports

In [ ]:
import plotly.express as px

def plot_attribute(df, attribute):
    
    df['time'] = df['time'].astype('str')
    
    df = df.groupby(['location', 'time', attribute], as_index = False)[attribute].size()
    
    print(df)
    
    # get coordinates for plotting
    df['X'] = df['location'].apply(set_value, args =(neighborhood_x, ))
    df['Y'] = df['location'].apply(set_value, args =(neighborhood_y, ))
    
    df['X'] = df['X'].apply(toggle)
    df['Y'] = df['Y'].apply(toggle)
    
    
    fig = px.scatter(df, 
                     x="X", 
                     y="Y", 
                     color = attribute, 
                     width=750, 
                     height=600, 
                     size = 'size',
                     color_continuous_scale = 'OrRd',
                     opacity = .6,
                     animation_frame="time")
                    #animation_group = 'location')

    #fig = px.scatter(width=750, height=600)#


    fig.add_layout_image(
    source=encoded_image,
                                xref='paper',
                                yref='paper',
                                x=0,
                                y=1,
                                sizex=1,
                                sizey=1,
                                sizing="stretch",
                                opacity=.55,
                                layer="below"
    )



    fig.update_xaxes(range=[0, 100])
    fig.update_yaxes(range=[0, 100])
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 50
    return fig    

In [ ]:
l = plot_attribute(damage_reports, 'shake_intensity')
l.show()

In [ ]:
l = plot_attribute(damage_reports, 'sewer_and_water')
l.show()

In [ ]:
l = plot_attribute(damage_reports, 'medical')
l.show()

In [ ]:
# before submission, format as html table
m = damage_reports.groupby(['location', 'time','buildings'], as_index = False)['buildings'].size()
m.sort_values(by = 'size',  ascending = False)

## Melt the data!!

In [ ]:
#damage_reports_long

damage_reports_long = pd.melt(damage_reports, id_vars=['time', 'X', 'Y', 'location'], var_name = 'category', value_name = 'value')

damage_summed = damage_reports_long.groupby(['location', 'time','X', 'Y', 'category'], as_index = False).agg({'value' : 'sum'})

In [ ]:
damage_summed['time'].unique().size

In [33]:
import plotly.express as px
fig = px.line(damage_summed, 
              x="time", 
              y="value", 
              color="category", 
              facet_col="location",
              facet_col_wrap=4, 
              height=1300, 
              width=1000,
              facet_row_spacing=.02)
#fig.update_yaxes(matches=None)

fig.show()